**Assignment 2**
----------------------------------
**Name** : *Ahmad Farhan* <br>
**Roll No.** : *i211366* <br>
**Section** : *A* <br>
__________________________________

### *Genetic Algorithm Solution to 3x3 Magic Puzzle Problem*
#### **Algorithm Parameters**
1. Population size (n): Number of individuals in population.
2. Elite size: Number of individuals with highest fitness scores to be transferred to the next generation automatically
3. Mutation Probability: Chances of Mutation occurance within individual
4. Crossover Functions: Both Are single point crossover functions
    - Partially Mapped Crossover
    - Inversion Sequence Crossover ([Research Paper](https://user.ceng.metu.edu.tr/~ucoluk/research/publications/tspnew.pdf))     


#### **Algorithm Definitions**

**Individual Representation**: <br>
Each individual is represented as a simple array with the sqaure in row major order i.e., [1,2,3,4,5,6,7,8,9]
<br>

**Fitness Function**: <br>
1. *Magic Number*($M$) is calculated using the formula $(n(n^2 + 1)/2)$<br>
2. *Sum*($S$) of each row, column and diagonals is calculated<br>
3. *Absolute Difference*($D$) of each $S$ is taken with $M$ i.e. $|S-M|$<br>
4. *Sum* of all $D$ of a square gives fitness of individual<br>

**Mating Pool Selection**: <br>
1. Individuals are selected using *tournament selection*. <br>
2. Mating pool size is set to half of population size. <br>
3. Two individuals with different fitnesses are randomly selected.<br>
4. The individual with better fitness is added to the pool.<br>
5. This process is repeadted till required pool size.<br>

**Elitism Selection**: <br>
A total of *elite_size* individuals with highest fitness value are selected and passed onto the next generation automatically.

**Crossover Function**: <br>
Both Crossover options are based on single point crossover and retain consistency of magic square problem i.e., do not allow duplicate genes within an individual. Cross over point $k$ is picked randomly.

1. Partially Mapper Crossover:<br>
$k$ values of the two parents, suppose $s$ and $t$, are swapped. $t_i$ is substituted into $s$ by swapping the $t_i$ value and the $s_i$ value of $s$.
e.g. Let $s$ = [5,7,1,3,6,4,2], $t$ = [4,6,2,7,3,1,5], the first swap would leave $s$ as [4,7,1,3,6,5,2]


2. Inversion Sequence Crossover:<br>
This crossover method is as propsed in this [research paper](https://user.ceng.metu.edu.tr/~ucoluk/research/publications/tspnew.pdf)
For each parent the inversion sequence array is calculated, which is a reversible representation of the same individual. Being reversible means that we can recalculate the original permutation from the inversion sequence. The inversion sequences are swapped upto k values. Then permutations are recalculated to complete crossover. This results in a crossover without generation of duplicate genes

**Mutation Function**: <br>
When probability condition is satisfied, two randomly selected genes of a individual are swapped within the same individual.

#### **Algorithm Analysis**

**Time Complexity**:<br>
Population Fitness: $O(n^2)$<br>
PMX-Crossover: $(O(kn))$<br>
Mutation: $O(1)$<br>
Single Iteration: $ O(kn^3) $<br>


**Space Complexity**:<br>
Population: $O(n)$<br>
PMX-Crossover: $(O(1))$<br>
Mutation: $O(1)$<br>
Overall: $ O(n) $<br>

In [11]:
import numpy as np 

In [12]:
# Function to calculate the magic number of rows,cols,diagonals
def magic_number(n): 
    return (n**3 + n)/2

# Function to swap two elements in a list
def swap(s,sval,tval):
    for i in range(len(s)):
        if s[i] == sval: s[i] = tval
        elif s[i] == tval: s[i] = sval

# Function to Calculate Inversion Sequence of a Permutation
def inverse_perm(perm):
    inv = []; n = len(perm)
    for i in range(1,n+1):
        count = 0; m = 0
        while m < n and perm[m] != i:
            if perm[m] > i:
                count += 1
            m += 1
        inv.append(count)
    return inv

# Function to Generate Permutation of Inversion Sequence 
def inv_to_perm(inv):
    n = len(inv); pos = [0] * n
    for i in range(n-1, -1, -1):
        pos[i] = inv[i]
        for m in range(i+1,n):
            if pos[m] >= inv[i]: pos[m] += 1
    perm = [0]*n
    for i in range(n): 
        perm[pos[i]] = i+1
    return perm

# Function to swap two elements of list
def flip(chrome, pos1, pos2):
    temp_element = chrome[pos1]
    chrome[pos1] = chrome[pos2]
    chrome[pos2] = temp_element
    return chrome

In [13]:
# Function to Calculate Fitness of Chromosome
def fitness(c):
    value = 0
    m = magic_number(3)
    for i in range(0, 9, 3):            # Difference of Row-wise Sum
        value += abs(m - sum(c[i:i+3]))
    for i in range(3):                  # Difference of Column-wise Sum
        value += abs(m - sum(c[i::3]))
    value += abs(m - sum(c[::4]))       # Difference Right Diagonal Sum
    value += abs(m - sum(c[6:1:-2]))    # Difference of Left Diagonal Sum 
    return int(value)

# Function to Select Mating Pool (Tournament Selection)
def selection(population):
    size = len(population)
    k = size//2; selected = []
    # Pick Two Random Individuals
    # Select Individual that has Better Fitness 
    while len(selected) < k:
        f1 = population[np.random.randint(0,size)]
        f2 = population[np.random.randint(0,size)]
        selected.append(f1 if f1[1] > f2[1] else f2)
    return selected

# Function for Partially Mapped Crossover
def crossover_pmx(s, t):
    k = np.random.randint(0,8)
    new_s = s.copy()
    new_t = t.copy()
    # K-elements of s are set to t in these positions
    # Swap s-value with t-value within s
    for i in range(k):  
        swap(new_s, new_s[i], new_t[i])
        swap(new_t, new_t[i], s[i])
    return new_s,new_t

# Function for Inversion Sequence Crossover
def crossover_perm(s, t):
    s_inv = inverse_perm(s)     # Inversion Sequence of s
    t_inv = inverse_perm(t)     # Inversion sequence of t
    k = np.random.randint(0,8)
    s_cpy = s_inv.copy()
    # Perform Crossover by Swapping Elements in Inversion Sequences
    s_inv[k:] = t_inv[k:]
    t_inv[k:] = s_cpy[k:]
    # Convert Modified Inversion Sequnces to Chromosomes
    s = inv_to_perm(s_inv)
    t = inv_to_perm(t_inv)
    return s,t

# Function to Mutate an individual
def mutate(chrome):
    pos1 = pos2 = None
    # Select two Genes of Individual and Swap
    while(True):
        pos1 = np.random.randint(0,8)
        pos2 = np.random.randint(0,8)
        if pos1 != pos2: break
    return flip(chrome,pos1,pos2)

# Function to Mutate n individuals based on probability
def mutation(children, m_prob):
    for child in children:
        if round(np.random.rand(),2) >= m_prob:
            mutate(child)
    return children

# Function to Select n-best Individuals
def elitism(population, elite_size):
    return [individual[0] for individual in population[:elite_size]]

In [14]:
# Function to generate the initial population of chromosomes
def generate_initial(n=9,l=9):
    base = np.array(range(1,l+1))   # List of Valid Genes [1-9]
    population = [] 
    for _ in range(n):
        # Create a chromosome by shuffling genes
        np.random.shuffle(base)
        population.append((base.tolist(),0))
    return population

# Function to calculate the fitness of each chromosome in the population
def calculate_fitness(population):
    if isinstance(population[0], tuple):
         # If the population consists of tuples (chromosome, fitness), update fitness values
        for idx,chrome in enumerate(population):
            population[idx] = (chrome[0],fitness(chrome[0]))
    else:# If the population consists only of chromosomes, calculate fitness and update
        population = list(population)
        for idx,chrome in enumerate(population):
            population[idx] = tuple((chrome, fitness(chrome)))
    return population

# Function to Display Chromosomes in Population
def display(p): 
    for i in p:print(i)
    print()

# Function to check if the target number of solutions is reached
def complete(found,target):  return bool(len(found) == target)

# Function to save the target chromosomes found in the population
def save_target(population, target, iter):
    for chrome in population:
        if chrome[1] == 0:              # If Fitness is 0 (target found)
            curr = tuple(chrome[0])
            if curr not in target:
                target.append(curr)     # Add target to list
                globals()['result_gens'].append(iter)

In [15]:
n = 9; result_gens = []  # List to store generation number where targets are found
p = generate_initial(n)  # Generate Initial Population of Chromosomes 
# Parameters for running Genetic Algorithm
target_count = 1; mutation_prob = 0.4
results = []; iterations = 10000; elite_size = 1


# Main Evolution until targets found or iter limit reached
for gen_count in range(iterations):
    p = calculate_fitness(p)            # Calculate Fitness of population
    p = sorted(p, key=lambda x:x[1])    # Sort Population based on fitness
    save_target(p, results, gen_count)  # Save Target Solutions if any found
    
    # Display Population Every 30 Iterations
    if gen_count == 0 or gen_count % 30 == 0: 
        print("Generation :", gen_count); display(p)    
    if complete(results, target_count):  break

    matingpool = selection(p)       # Select individuals for mating
    next_gen = elitism(p, elite_size)     # Preserve top individuals
    
    # Generate offspring until the population size is reached
    while(len(next_gen) < n):
        parent1 = p[np.random.randint(0,n)]  # Select parent 1 randomly
        parent2 = p[np.random.randint(0,n)]  # Select parent 2 randomly
        # Perform crossover and mutation to generate children
        children = crossover_perm(parent1[0], parent2[0])
        children = mutation(children, mutation_prob)
        next_gen.extend(children)           # Add children to generation
    # gen_count += 1
    p = next_gen  # Update the population with the new generation

# Display the Results    
if len(results) > 0:
    print("Result Generation :", result_gens)
    print("Result : ",end='')
    display(results)
else: print("Target Not Found within Generations Limit")

Generation : 0
([4, 3, 7, 5, 1, 9, 8, 6, 2], 21)
([4, 1, 6, 8, 7, 2, 5, 3, 9], 24)
([5, 4, 8, 9, 7, 6, 2, 3, 1], 24)
([9, 6, 1, 7, 5, 3, 8, 2, 4], 24)
([6, 2, 7, 4, 9, 8, 5, 1, 3], 27)
([3, 2, 1, 5, 7, 4, 6, 8, 9], 27)
([6, 7, 3, 8, 1, 4, 5, 9, 2], 28)
([4, 1, 2, 5, 8, 3, 7, 9, 6], 29)
([1, 4, 2, 5, 7, 9, 3, 6, 8], 32)

Generation : 30
([7, 1, 4, 3, 6, 9, 5, 8, 2], 6)
([7, 1, 4, 3, 6, 8, 5, 9, 2], 8)
([5, 3, 7, 4, 1, 8, 6, 9, 2], 16)
([5, 3, 4, 1, 8, 6, 9, 7, 2], 18)
([6, 7, 4, 3, 1, 8, 5, 9, 2], 21)
([1, 7, 4, 3, 6, 9, 5, 8, 2], 24)
([7, 3, 4, 6, 9, 5, 1, 8, 2], 24)
([5, 1, 3, 9, 7, 8, 4, 6, 2], 26)
([4, 1, 6, 8, 5, 3, 9, 7, 2], 29)

Generation : 60
([8, 1, 6, 3, 4, 7, 5, 9, 2], 5)
([9, 8, 6, 3, 4, 7, 5, 1, 2], 20)
([1, 9, 6, 3, 4, 7, 5, 8, 2], 22)
([1, 8, 6, 3, 4, 7, 5, 9, 2], 22)
([1, 8, 5, 3, 4, 7, 6, 9, 2], 24)
([8, 6, 7, 3, 4, 1, 5, 9, 2], 26)
([7, 5, 9, 3, 4, 1, 6, 8, 2], 26)
([1, 7, 9, 3, 4, 5, 6, 8, 2], 28)
([8, 6, 7, 3, 4, 1, 9, 5, 2], 30)

Result Generation : [84]
Result : (